<a href="https://colab.research.google.com/github/elangbijak4/blockchain-codes-use-generative-AI/blob/main/Rev3_Infrastruktur_Blockchain_Dengan_Implementasi_ERC721.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import hashlib
import time
import json
import os

## Kelas Block

In [30]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

    @staticmethod
    def calculate_hash(index, previous_hash, timestamp, data):
        value = f"{index}{previous_hash}{timestamp}{data}"
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    @staticmethod
    def create_genesis_block():
        return Block(0, "0", int(time.time()), "Genesis Block", Block.calculate_hash(0, "0", int(time.time()), "Genesis Block"))

    def to_dict(self):
        return {
            "index": self.index,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp,
            "data": self.data,
            "hash": self.hash
        }

## Kelas Smart Contract

In [31]:
class SmartContract:
    def __init__(self, code):
        self.code = code
        self.storage = {}

    def execute(self, blockchain, **kwargs):
        local_context = {
            'blockchain': blockchain,
            'storage': self.storage
        }
        local_context.update(kwargs)
        exec(self.code, {}, local_context)
        self.storage = local_context['storage']

Implementasi ERC721 ke dalam Kelas ERC721

In [32]:
class ERC721:
    def __init__(self):
        self.tokens = {}
        self.token_owners = {}
        self.token_metadata = {}
        self._token_owner = {}  # tokenId => owner
        self._owned_tokens = {}  # owner => list of tokenIds
        self._token_approvals = {}  # tokenId => approved address
        self._operator_approvals = {}  # owner => (operator => approved)

    def balance_of(self, owner):
        if owner in self._owned_tokens:
            return len(self._owned_tokens[owner])
        return 0

    def owner_of(self, tokenId):
        return self._token_owner.get(tokenId, None)

    def transfer_from(self, from_address, to_address, tokenId):
        if tokenId not in self.tokens or self.tokens[tokenId]["owner"] != from_address:
            raise Exception("Transfer not allowed")
        self.tokens[tokenId]["owner"] = to_address
        self.token_owners[from_address].remove(tokenId)
        if to_address in self.token_owners:
            self.token_owners[to_address].append(tokenId)
        else:
            self.token_owners[to_address] = [tokenId]

    def approve(self, to, tokenId):
        owner = self.owner_of(tokenId)
        if owner:
            self._token_approvals[tokenId] = to

    def set_approval_for_all(self, owner, operator, approved):
        if owner in self._operator_approvals:
            self._operator_approvals[owner][operator] = approved
        else:
            self._operator_approvals[owner] = {operator: approved}

    def is_approved_for_all(self, owner, operator):
        return self._operator_approvals.get(owner, {}).get(operator, False)

    def _is_approved_or_owner(self, spender, tokenId):
        owner = self.owner_of(tokenId)
        return spender == owner or \
               self._token_approvals.get(tokenId) == spender or \
               self.is_approved_for_all(owner, spender)

    def _transfer(self, from_address, to_address, tokenId):
        # Remove token from the current owner's list
        if from_address in self._owned_tokens:
            self._owned_tokens[from_address].remove(tokenId)

        # Add token to the new owner's list
        if to_address in self._owned_tokens:
            self._owned_tokens[to_address].append(tokenId)
        else:
            self._owned_tokens[to_address] = [tokenId]

        # Change token owner
        self._token_owner[tokenId] = to_address

        # Clear approval
        if tokenId in self._token_approvals:
            del self._token_approvals[tokenId]

    def mint(self, to, tokenId, contract_address, uri):
        if tokenId in self.tokens:
            raise Exception("Token already exists")
        self.tokens[tokenId] = {
            "tokenId": tokenId,
            "owner": to,
            "contract_address": contract_address,
            "uri": uri
        }
        self.token_owners[to] = self.token_owners.get(to, []) + [tokenId]
        self.token_metadata[tokenId] = {
            "owner": to,
            "contract_address": contract_address,
            "uri": uri
        }

    def burn(self, tokenId):
        owner = self.owner_of(tokenId)
        if owner:
            self._owned_tokens[owner].remove(tokenId)
            del self._token_owner[tokenId]
            if tokenId in self._token_approvals:
                del self._token_approvals[tokenId]

    def get_token_metadata(self, tokenId):
        return self.token_metadata.get(tokenId, None)

### Integrasi ERC721 dengan Blockchain

In [34]:
class Blockchain:
    def __init__(self):
        self.chain = [Block.create_genesis_block()]
        self.erc721 = ERC721()  # Tambahkan inisialisasi ERC721
        self.smart_contracts = self.load_smart_contracts()  # Load smart contracts from file

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        latest_block = self.get_latest_block()
        new_index = latest_block.index + 1
        new_timestamp = int(time.time())
        new_hash = Block.calculate_hash(new_index, latest_block.hash, new_timestamp, data)
        new_block = Block(new_index, latest_block.hash, new_timestamp, data, new_hash)
        self.chain.append(new_block)
        return new_block

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]

            if current_block.hash != Block.calculate_hash(current_block.index, current_block.previous_hash, current_block.timestamp, current_block.data):
                return False

            if current_block.previous_hash != previous_block.hash:
                return False

        return True

    def mint_nft(self, to, uri):
        tokenId = hashlib.sha256(uri.encode()).hexdigest()  # Generate tokenId from URI
        contract_address = self.store_smart_contract("ERC721 Contract Code")
        self.erc721.mint(to, tokenId, contract_address, uri)
        block = self.add_block(f"Minted NFT {tokenId} to {to}, contract {contract_address}, uri {uri}")
        return block.hash

    def transfer_nft(self, from_address, to_address, tokenId):
        self.erc721.transfer_from(from_address, to_address, tokenId)
        block = self.add_block(f"Transferred NFT {tokenId} from {from_address} to {to_address}")
        return block.hash

    def store_smart_contract(self, code):
        contract = SmartContract(code)
        block = self.add_block(f"Smart contract code: {code}")
        self.smart_contracts[block.hash] = code  # Store code directly
        self.save_smart_contracts()
        return block.hash

    def execute_smart_contract_from_block(self, block_hash, **kwargs):
        # Temukan blok berdasarkan hash
        block = next((blk for blk in self.chain if blk.hash == block_hash), None)
        if block and block.data.startswith("Smart contract code:"):
            # Ekstrak kode smart contract dari data blok
            code = block.data[len("Smart contract code: "):]
            contract = SmartContract(code)
            contract.execute(self, **kwargs)
        else:
            print("Smart contract not found in the specified block")

    def load_smart_contracts(self):
        if os.path.exists('smart_contracts.json'):
            with open('smart_contracts.json', 'r') as f:
                return json.load(f)
        return {}

    def save_smart_contracts(self):
        with open('smart_contracts.json', 'w') as f:
            json.dump(self.smart_contracts, f)

    def to_json(self):
        return json.dumps([block.to_dict() for block in self.chain], indent=4)


## Demo Pembuatan Smart Contract

In [37]:
if __name__ == "__main__":
    # Membuat blockchain baru
    my_blockchain = Blockchain()

    # Menambahkan blok baru
    my_blockchain.add_block("Block 1 Data")
    my_blockchain.add_block("Block 2 Data")

    print("Blockchain valid:", my_blockchain.is_chain_valid())

    # Membuat smart contract sederhana
    smart_contract_code = """
def contract_function(storage):
    storage['counter'] = storage.get('counter', 0) + 1
    print(f"Counter value: {storage['counter']}")
contract_function(storage)
"""

    # Menyimpan smart contract ke blockchain
    block_hash = my_blockchain.store_smart_contract(smart_contract_code)
    print(f"Smart contract stored in block with hash: {block_hash}")

    # Menjalankan smart contract dari blockchain
    my_blockchain.execute_smart_contract_from_block(block_hash)

    # Menjalankan kembali smart contract untuk melihat perubahan state
    my_blockchain.execute_smart_contract_from_block(block_hash)

    # Mencetak isi blockchain dalam format JSON
    print("Blockchain content in JSON:")
    print(my_blockchain.to_json())

Blockchain valid: True
Smart contract stored in block with hash: b35d347111914447d20187e1c3010ec01f25d953c758dfefb3424926e15e494c
Counter value: 1
Counter value: 1
Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1716916868,
        "data": "Genesis Block",
        "hash": "4e700c5f8f2916d78f83b1383590fd9f55a3fcc90e80279815539aac15283a0f"
    },
    {
        "index": 1,
        "previous_hash": "4e700c5f8f2916d78f83b1383590fd9f55a3fcc90e80279815539aac15283a0f",
        "timestamp": 1716916868,
        "data": "Block 1 Data",
        "hash": "38f7d490495e0aa70df27b3c2cbefd115913f082db810e12792dcc3b0d1f67c7"
    },
    {
        "index": 2,
        "previous_hash": "38f7d490495e0aa70df27b3c2cbefd115913f082db810e12792dcc3b0d1f67c7",
        "timestamp": 1716916868,
        "data": "Block 2 Data",
        "hash": "db5809f6f4ba20e480486292774f83b94064f2f05dbb2cda8af7454e29c809e2"
    },
    {
        "index": 3,
        "previous_ha

## Demo Penggunaan ERC-721

In [38]:
if __name__ == "__main__":
    # Membuat blockchain baru
    #my_blockchain = Blockchain()

    # Menambahkan blok baru
    my_blockchain.add_block("Block 1 Data")
    my_blockchain.add_block("Block 2 Data")

    print("Blockchain valid:", my_blockchain.is_chain_valid())

    # Membuat NFT baru
    owner_address = "owner_wallet_address"
    asset_uri = "http://example.com/my-nft"
    mint_block_hash = my_blockchain.mint_nft(owner_address, asset_uri)
    print(f"NFT minted in block with hash: {mint_block_hash}")

    # Transfer NFT
    new_owner_address = "new_owner_wallet_address"
    token_id = hashlib.sha256(asset_uri.encode()).hexdigest()
    transfer_block_hash = my_blockchain.transfer_nft(owner_address, new_owner_address, token_id)
    print(f"NFT transferred in block with hash: {transfer_block_hash}")

    # Mencetak isi blockchain dalam format JSON
    print("Blockchain content in JSON:")
    print(my_blockchain.to_json())


Blockchain valid: True
NFT minted in block with hash: 8166b684241adc2dd3c220dccb3ed84707e6fe7fda6d377d3d095ad723a16cce
NFT transferred in block with hash: 1c3ebcb64dac19585fae1d9cdefb028522ce46202ffc350912b607453a045272
Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1716916868,
        "data": "Genesis Block",
        "hash": "4e700c5f8f2916d78f83b1383590fd9f55a3fcc90e80279815539aac15283a0f"
    },
    {
        "index": 1,
        "previous_hash": "4e700c5f8f2916d78f83b1383590fd9f55a3fcc90e80279815539aac15283a0f",
        "timestamp": 1716916868,
        "data": "Block 1 Data",
        "hash": "38f7d490495e0aa70df27b3c2cbefd115913f082db810e12792dcc3b0d1f67c7"
    },
    {
        "index": 2,
        "previous_hash": "38f7d490495e0aa70df27b3c2cbefd115913f082db810e12792dcc3b0d1f67c7",
        "timestamp": 1716916868,
        "data": "Block 2 Data",
        "hash": "db5809f6f4ba20e480486292774f83b94064f2f05dbb2cda8af7454e29c809e